In [2]:
import pandas as pd
import numpy as np
import ast
pd.set_option("display.width", 170)
pd.set_option("display.max_columns", 200)

In [3]:
google_restaurants = pd.read_csv("enriched_restaurants.csv")
print(google_restaurants.shape)
print(google_restaurants.info())
print(google_restaurants.head())

(6998, 35)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6998 entries, 0 to 6997
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   business_id               6998 non-null   object 
 1   displayName               6998 non-null   object 
 2   address                   6998 non-null   object 
 3   price_level               6026 non-null   object 
 4   price_range               5574 non-null   object 
 5   rating                    6949 non-null   float64
 6   regular_opening_hours     6998 non-null   object 
 7   user_rating_count         6949 non-null   float64
 8   number_of_photos          6998 non-null   int64  
 9   types                     6998 non-null   object 
 10  takeout                   6998 non-null   object 
 11  delivery                  6998 non-null   object 
 12  dine_in                   6998 non-null   object 
 13  reservable                6998 non-null   object 
 1

In [4]:
# remove empty columns (payment_options, ev_charge_options, fuel_options)
google_restaurants = google_restaurants.drop(columns=["payment_options", "ev_charge_options", "fuel_options"])
print(google_restaurants.shape)

(6998, 32)


In [5]:
restaurants = pd.read_csv("restaurants.csv", index_col=0)
print(restaurants.shape)
print(restaurants.info())
print(restaurants.head())

(8069, 48)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8069 entries, 0 to 8068
Data columns (total 48 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      8069 non-null   object 
 1   city                             8069 non-null   object 
 2   postal_code                      8069 non-null   int64  
 3   categories                       8069 non-null   object 
 4   stars                            8069 non-null   float64
 5   name                             8069 non-null   object 
 6   RestaurantsTakeOut               7471 non-null   object 
 7   BusinessAcceptsCreditCards       7181 non-null   object 
 8   RestaurantsDelivery              7149 non-null   object 
 9   RestaurantsPriceRange2           6581 non-null   float64
 10  RestaurantsReservations          6376 non-null   object 
 11  HasTV                            6366 non-null   object 
 12  OutdoorSe

In [6]:
# combine both dataframes via outer join on business_id
restaurants_merged = pd.merge(restaurants, google_restaurants, how="outer", on="business_id")
print(restaurants_merged.shape)
print(restaurants_merged.info())
print(restaurants_merged.head())
print(restaurants_merged.describe(include="all"))

(8069, 79)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8069 entries, 0 to 8068
Data columns (total 79 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      8069 non-null   object 
 1   city                             8069 non-null   object 
 2   postal_code                      8069 non-null   int64  
 3   categories                       8069 non-null   object 
 4   stars                            8069 non-null   float64
 5   name                             8069 non-null   object 
 6   RestaurantsTakeOut               7471 non-null   object 
 7   BusinessAcceptsCreditCards       7181 non-null   object 
 8   RestaurantsDelivery              7149 non-null   object 
 9   RestaurantsPriceRange2           6581 non-null   float64
 10  RestaurantsReservations          6376 non-null   object 
 11  HasTV                            6366 non-null   object 
 12  OutdoorSe

In [7]:
print(restaurants_merged.columns)

Index(['business_id', 'city', 'postal_code', 'categories', 'stars', 'name', 'RestaurantsTakeOut', 'BusinessAcceptsCreditCards', 'RestaurantsDelivery',
       'RestaurantsPriceRange2', 'RestaurantsReservations', 'HasTV', 'OutdoorSeating', 'RestaurantsGoodForGroups', 'WiFi', 'GoodForKids', 'BikeParking',
       'RestaurantsAttire', 'Caters', 'BusinessParking', 'NoiseLevel', 'Ambience', 'GoodForMeal', 'RestaurantsTableService', 'WheelchairAccessible', 'HappyHour',
       'Alcohol', 'DogsAllowed', 'BusinessAcceptsBitcoin', 'BYOB', 'Corkage', 'DriveThru', 'BestNights', 'CoatCheck', 'ByAppointmentOnly', 'GoodForDancing', 'Smoking',
       'BYOBCorkage', 'Music', 'AgesAllowed', 'RestaurantsCounterService', 'Open24Hours', 'AcceptsInsurance', 'DietaryRestrictions', 'dist_highway',
       'adjusted_gross_income', 'rural_urban_continuum_code_2023', 'unemployment_rate_2023', 'displayName', 'address', 'price_level', 'price_range', 'rating',
       'regular_opening_hours', 'user_rating_count', 'numb

In [8]:
# remove address column: address as a feature is captured by latitude and longitude
# remove displayName: this is the same as name
# remove regular_opening_hours: it is complicated to use this data, as there are multiple opening hours for different days
restaurants_merged = restaurants_merged.drop(columns=["address", "displayName", "regular_opening_hours"])

In [9]:
# get restaurantpricerange2 and price_level from restaurants_merged
a = restaurants_merged[["RestaurantsPriceRange2", "price_level"]]
print(a.groupby(["RestaurantsPriceRange2", "price_level"]).size())
# based on the grouping below, we can see that there are 4 price levels for both columns
# we can use the price_level column to fill in the missing values in RestaurantsPriceRange2

RestaurantsPriceRange2  price_level               
1.0                     PRICE_LEVEL_EXPENSIVE            3
                        PRICE_LEVEL_INEXPENSIVE       1925
                        PRICE_LEVEL_MODERATE           424
                        PRICE_LEVEL_VERY_EXPENSIVE       1
2.0                     PRICE_LEVEL_EXPENSIVE           38
                        PRICE_LEVEL_INEXPENSIVE        623
                        PRICE_LEVEL_MODERATE          1887
                        PRICE_LEVEL_VERY_EXPENSIVE       2
3.0                     PRICE_LEVEL_EXPENSIVE           77
                        PRICE_LEVEL_INEXPENSIVE          8
                        PRICE_LEVEL_MODERATE            58
                        PRICE_LEVEL_VERY_EXPENSIVE       7
4.0                     PRICE_LEVEL_EXPENSIVE            3
                        PRICE_LEVEL_INEXPENSIVE          6
                        PRICE_LEVEL_MODERATE             6
                        PRICE_LEVEL_VERY_EXPENSIVE       9
dtype

In [10]:
# convert price_level values to RestaurantsPriceRange2 values
restaurants_merged["price_level"] = restaurants_merged["price_level"].map({"PRICE_LEVEL_INEXPENSIVE": 1, "PRICE_LEVEL_MODERATE": 2, "PRICE_LEVEL_EXPENSIVE": 3, 
                                                                           "PRICE_LEVEL_VERY_EXPENSIVE": 4})
# fill in missing values in RestaurantsPriceRange2 with values from price_level
restaurants_merged["RestaurantsPriceRange2"] = restaurants_merged["RestaurantsPriceRange2"].fillna(restaurants_merged["price_level"])

In [11]:
# # remove price_level column
restaurants_merged = restaurants_merged.drop(columns=["price_level"])
# rename RestaurantsPriceRange2 to price_level
restaurants_merged = restaurants_merged.rename(columns={"RestaurantsPriceRange2": "price_level"})

In [12]:
# get price_range_low and price_range_high from restaurants_merged
# restaurants_merged['price_range_low'] = restaurants_merged['price_range']


In [13]:
print(type(restaurants_merged["price_range"][1]))

<class 'str'>


In [14]:
# rename rating as google_rating
restaurants_merged = restaurants_merged.rename(columns={"rating": "google_rating", "stars": "yelp_rating"})

In [15]:
# convert user_rating_count to integer (nan to remain as nan)
restaurants_merged["user_rating_count"] = restaurants_merged["user_rating_count"].astype("Int64")

In [16]:
# convert columns to boolean
bool_cols = ["takeout", "delivery", "dine_in", "reservable", "serves_breakfast", "serves_lunch", "serves_dinner", "serves_brunch", "serves_beer",
             "serves_wine", "serves_cocktails", "serves_dessert", "serves_coffee", "outdoor_seating", "live_music", "allows_dogs", "good_for_children",
             "good_for_groups", "good_for_watching_sports", "menu_for_children"]
for col in bool_cols:
    restaurants_merged[col] = restaurants_merged[col].replace({"Yes": True, "No": False})


In [17]:
# fillna for RestaurantTakeOut
restaurants_merged["RestaurantsTakeOut"] = restaurants_merged["RestaurantsTakeOut"].fillna(restaurants_merged["takeout"])
# fillna for RestaurantsDelivery
restaurants_merged["RestaurantsDelivery"] = restaurants_merged["RestaurantsDelivery"].fillna(restaurants_merged["delivery"])
# fillna for RestaurantsReservations
restaurants_merged["RestaurantsReservations"] = restaurants_merged["RestaurantsReservations"].fillna(restaurants_merged["reservable"])

# remove takeout, delivery, reservable columns
restaurants_merged = restaurants_merged.drop(columns=["takeout", "delivery", "reservable"])

In [18]:
# convert GoodForMeal to list of words
restaurants_merged["GoodForMeal"] = restaurants_merged["GoodForMeal"].str.split(", ")
# fillna as list
restaurants_merged["GoodForMeal"] = restaurants_merged["GoodForMeal"].apply(lambda x: x if isinstance(x, list) else [])

# add "breakfast" to GoodForMeal if serves_breakfast is True and there is no "breakfast" in GoodForMeal, and so on for lunch, dinner, brunch
restaurants_merged["GoodForMeal"] = restaurants_merged.apply(lambda x: x["GoodForMeal"] + ["breakfast"] if (x["serves_breakfast"] == True and "breakfast" not in x["GoodForMeal"]) else x["GoodForMeal"], axis=1)
restaurants_merged["GoodForMeal"] = restaurants_merged.apply(lambda x: x["GoodForMeal"] + ["lunch"] if (x["serves_lunch"] == True and "lunch" not in x["GoodForMeal"]) else x["GoodForMeal"], axis=1)
restaurants_merged["GoodForMeal"] = restaurants_merged.apply(lambda x: x["GoodForMeal"] + ["dinner"] if (x["serves_dinner"] == True and "dinner" not in x["GoodForMeal"]) else x["GoodForMeal"], axis=1)
restaurants_merged["GoodForMeal"] = restaurants_merged.apply(lambda x: x["GoodForMeal"] + ["brunch"] if (x["serves_brunch"] == True and "brunch" not in x["GoodForMeal"]) else x["GoodForMeal"], axis=1)



In [19]:
# fillna for Alcohol based on if there are True in serves_beer, serves_wine, serves_cocktails
restaurants_merged["Alcohol"] = restaurants_merged["Alcohol"].fillna(restaurants_merged["serves_beer"] | restaurants_merged["serves_wine"] | restaurants_merged["serves_cocktails"])

In [20]:
# fillna for OutdoorSeating based on if there are True in outdoor_seating
restaurants_merged["OutdoorSeating"] = restaurants_merged["OutdoorSeating"].fillna(restaurants_merged["outdoor_seating"])
# fillna for Music based on if there are True in live_music
restaurants_merged["Music"] = restaurants_merged["Music"].fillna(restaurants_merged["live_music"])
# fillna for DogsAllowed based on if there are True in allows_dogs
restaurants_merged["DogsAllowed"] = restaurants_merged["DogsAllowed"].fillna(restaurants_merged["allows_dogs"])
# fillna for GoodForKids
restaurants_merged["GoodForKids"] = restaurants_merged["GoodForKids"].fillna(restaurants_merged["good_for_children"])
# fillna for RestaurantsGoodForGroups
restaurants_merged["RestaurantsGoodForGroups"] = restaurants_merged["RestaurantsGoodForGroups"].fillna(restaurants_merged["good_for_groups"])

# remove outdoor_seating, live_music, allows_dogs, good_for_children, good_for_groups columns
restaurants_merged = restaurants_merged.drop(columns=["outdoor_seating", "live_music", "allows_dogs", "good_for_children", "good_for_groups"])

In [21]:
# convert to string to dict
restaurants_merged["editorial_summary"] = restaurants_merged["editorial_summary"].astype(str)
restaurants_merged["editorial_summary"] = restaurants_merged["editorial_summary"].apply(lambda x: {'text': '', 'languageCode': ''} if x == "nan" else ast.literal_eval(x))
print(restaurants_merged["editorial_summary"].head())

0                     {'text': '', 'languageCode': ''}
1    {'text': 'Chic sushi & raw bar in modern surro...
2    {'text': 'Classic Korean dishes & beer in simp...
3    {'text': 'Long-standing casual pizzeria that b...
4    {'text': 'Easygoing restaurant preparing gener...
Name: editorial_summary, dtype: object


In [22]:
restaurants_merged["editorial_summary_language"] = restaurants_merged["editorial_summary"].apply(lambda x: x.get("languageCode"))
print(restaurants_merged["editorial_summary_language"].head())
restaurants_merged["editorial_summary"] = restaurants_merged["editorial_summary"].apply(lambda x: x.get("text"))
print(restaurants_merged["editorial_summary"].head())

0      
1    en
2    en
3    en
4    en
Name: editorial_summary_language, dtype: object
0                                                     
1    Chic sushi & raw bar in modern surrounds with ...
2    Classic Korean dishes & beer in simple digs in...
3    Long-standing casual pizzeria that backs up it...
4    Easygoing restaurant preparing generous burger...
Name: editorial_summary, dtype: object


In [23]:
# get key value pairs from parking_options
restaurants_merged["parking_options"] = restaurants_merged["parking_options"].astype(str)
restaurants_merged["parking_options"] = restaurants_merged["parking_options"].apply(lambda x: {} if x == "nan" else ast.literal_eval(x))

def get_parking_options(x):
    options = []
    items = x.items()
    for key, value in items:
        if value == True:
            options.append(key)
        else:
            options.append('no ' + key)
    return options


restaurants_merged["parking_options"] = restaurants_merged["parking_options"].apply(lambda x: get_parking_options(x))
print(restaurants_merged["parking_options"].head())

0                                                   []
1    [no freeParkingLot, no paidParkingLot, no free...
2                                    [no valetParking]
3    [freeStreetParking, paidStreetParking, no vale...
4    [freeParkingLot, freeStreetParking, no valetPa...
Name: parking_options, dtype: object


In [24]:
print(restaurants_merged.shape)
print(restaurants_merged.info())
print(restaurants_merged.head())
print(restaurants_merged.describe(include="all"))
# get null values
pd.set_option("display.max_rows", None)
print(restaurants_merged.isnull().sum())
pd.set_option("display.max_rows", 10)

(8069, 68)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8069 entries, 0 to 8068
Data columns (total 68 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      8069 non-null   object 
 1   city                             8069 non-null   object 
 2   postal_code                      8069 non-null   int64  
 3   categories                       8069 non-null   object 
 4   yelp_rating                      8069 non-null   float64
 5   name                             8069 non-null   object 
 6   RestaurantsTakeOut               7989 non-null   object 
 7   BusinessAcceptsCreditCards       7181 non-null   object 
 8   RestaurantsDelivery              7942 non-null   object 
 9   price_level                      7529 non-null   float64
 10  RestaurantsReservations          7838 non-null   object 
 11  HasTV                            6366 non-null   object 
 12  OutdoorSe

In [25]:
# save to csv
restaurants_merged.to_csv("restaurants_merged.csv")